In [61]:
import requests

In [62]:
import cloudscraper
import geopandas as gpd
import pandas as pd

## Open State data

In [63]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)

## Get Homeownership Data

In [64]:
scraper = cloudscraper.create_scraper()

#### Get 2024 ownership rate

In [65]:
url = "https://datawrapper.dwcdn.net/oPsf9/6/dataset.csv"
r = scraper.get(url, headers=headers)
csv_string_split = r.text.split("\n")
list_of_records = [record.split(",") for record in csv_string_split]
ownership_per_state_df = pd.DataFrame(list_of_records[1:], columns=list_of_records[0])

In [67]:
ownership_per_state_df["NAME"] = ownership_per_state_df["State"]
ownership_per_state_df["ownership_rate_2024"] = (
    ownership_per_state_df["Owner-occupied Current Year"]
    .str.replace("%", "")
    .astype(float)
)
ownership_per_state_df["ownership_rate_2023"] = (
    ownership_per_state_df["Owner-occupied Last Year"]
    .str.replace("%", "")
    .astype(float)
)
ownership_per_state_df["renter_rate_2024"] = (
    ownership_per_state_df["Renter-occupied Current Year"]
    .str.replace("%", "")
    .astype(float)
)
ownership_per_state_df["renter_rate_2023"] = (
    ownership_per_state_df["Renter-occupied Last Year"]
    .str.replace("%", "")
    .astype(float)
)

In [68]:
ownership_per_state_df["ownership_diff"] = (
    ownership_per_state_df["ownership_rate_2024"]
    - ownership_per_state_df["ownership_rate_2023"]
)
ownership_per_state_df["renter_diff"] = (
    ownership_per_state_df["renter_rate_2024"]
    - ownership_per_state_df["renter_rate_2023"]
)

In [69]:
ownership_per_state_df = ownership_per_state_df[
    [
        "NAME",
        "ownership_rate_2024",
        "ownership_rate_2023",
        "ownership_diff",
        "renter_rate_2024",
        "renter_rate_2023",
        "renter_diff",
    ]
]

## Merge Data

In [71]:
housing_gdf = states_df.merge(ownership_per_state_df, on="NAME", how="inner").dropna()

In [72]:
housing_gdf = housing_gdf[
    [
        "NAME",
        "geometry",
        "ownership_rate_2024",
        "ownership_rate_2023",
        "ownership_diff",
        "renter_rate_2024",
        "renter_rate_2023",
        "renter_diff",
    ]
]

In [74]:
housing_gdf = housing_gdf.to_crs(9311)
housing_gdf.to_file("data/housing.gpkg")